Dealing with scoring. Will convert to scripts later 


- must check that size of y and size of scoring file the same
- 

# This jupyter file has code for all graphs and tables at the member level

In [1]:
import pandas as pd 
import numpy as np
import os 
import sys 
job = '71'
from code.data_processing import get_data
y = None

In [2]:
from code.data_processing import get_all_datasets
all_datasets = get_all_datasets()
datasets = {}
for d in all_datasets:
    datasets[d] = get_data(d)

First some kind of measure of how dominant a single classifier is 

# Functions to deal with dataset

In [3]:
# load a single seed of a task with ypred vs observation 
def load_scoring(fn : str) -> np.array: 
    assert('.csv' in fn)
    x = pd.read_csv(fn).transpose().to_numpy()
    assert(len(x.shape) == 2)
    return x 

In [11]:
# load each scoring dataframe for each seed
def load_all_scoring(path : str, training=True) -> np.array: 
    F = []
    for x in os.listdir(path):
        if (not '.csv' in x):
            continue 
        if training:
            if (not 'training' in x):
                continue
        else:
            if (not 'test' in x):
                continue
        F.append(load_scoring(path+'/'+x))
                
    return np.array(F)

In [21]:
# get accuracy of a single member vector on y
def acc_member(X : np.array) -> np.array: # gets the same X for each m ember 
    print(X)
    assert(len(X.shape) == 1)
    assert(len(y) == len(X))
    
    correct = 0
    for i in range(len(y)):
        if y[i] == X[i]: # fail here 
            correct += 1
    return correct / len(X)

In [6]:
# get accuracy of a single seed
def acc_seed(X : np.array) -> np.array:
    assert(len(X.shape) == 2)
    r = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        r[i] = acc_member(X[i])
    return r

In [7]:
# get accuracy over all seeds
def acc_task(X : np.array) -> list: # list not dict for now 
    assert(len(X.shape) == 3)
    r = []
    for i in range(X.shape[0]):
        r.append(acc_seed(X[i]))
    return r

# Graphing functions 

# Now loading in 

In [8]:
scoring_task_1 = load_all_scoring('results_file/4080640/1/scoring/')
print(f'Shape of all scores in task  = {scoring_task_1.shape}, (# seeds, # members in ensemble, # observatins)')
acc_task_1 = np.array(acc_task(scoring_task_1))
print(f'Shape for each seed = {acc_task_1[0].shape}')
print(f'Shape for acc over seeds = {acc_task_1.shape}') # might fail on ragged array

Shape of all scores in task  = (30, 5, 207), (# seeds, # members in ensemble, # observatins)


TypeError: object of type 'NoneType' has no len()

In [ ]:
acc_task_1

In [ ]:
#df_ensemble = pd.read_csv(f'results_file/{job}/1/bagboost_experiment_{job}_task_1_cleveland.csv').describe()
#df_ensemble

In [ ]:
#mean_acc = df_ensemble.loc['mean', 'full_acc']
#mean_acc

# Looking at how much each solution dominates
Ideally need to find a measure which doesn't depend on ensemble size... variance?
I think mean across seeds, variance accross accuracy of members

In [ ]:
mnvr = np.mean(np.var(acc_task_1, axis=1))
mnvr

In [ ]:
sys.path

# Preparing model vs dataset table

In [15]:
from sklearn.model_selection import train_test_split
def get_ytrain(j, t, training=True):
    dns = get_all_datasets()
    with open(f'results_file/{job}/{task}/{job}_{task}_info.txt') as f:
        data = f.read()
    for dn in dns:
        if dn in data:
            X, y = get_data(dn)
            seed = 169 * int(task)
            _, _, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed, stratify=y)
            if training:
                return y_train
            else:
                return y_test


In [22]:
acc_all_tasks = []
max_nmembers = -1
for task in os.listdir(f'results_file/{job}'):
    # Not all done
    if len(os.listdir(f'results_file/{job}/{task}')) <= 1:
        continue 
    # Otherwise load it 
    scoring_task = load_all_scoring(f'results_file/{job}/{task}/scoring/', training=False) # (# seeds, # members in ensemble, # observatins)
    y = get_ytrain(job, task, training=False)
    acc_taskx = np.array(acc_task(scoring_task))
    acc_all_tasks.append(acc_taskx)
    if acc_taskx.shape[1] > max_nmembers:
        max_nmembers = acc_taskx.shape[1]

[1 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 1 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0
 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0]
[1 1 0 1 0 1 1 1 1 0 0 1 0 1 0 1 1 1 0 0 1 1 0 0 0 1 1 1 1 0 1 0 1 0 1 1 0
 0 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 0 0 0 1 0 1 0 0 1 0 1 1 1 0 1 1 1 0 1 1
 0 0 1 0 1 0 1 1 1 1 1 0 1 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0 1 0 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 0 0 0 1 1 0 1 1 1
 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1
 1 1 1 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1]
[1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 0 0 0 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 